### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [328]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data['Age'] = purchase_data['Age'].astype(float)

purchase_data.head(25)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20.0,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40.0,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24.0,Male,92,Final Critic,4.88
3,3,Chamassasya86,24.0,Male,100,Blindscythe,3.27
4,4,Iskosia90,23.0,Male,131,Fury,1.44
5,5,Yalae81,22.0,Male,81,Dreamkiss,3.61
6,6,Itheria73,36.0,Male,169,"Interrogator, Blood Blade of the Queen",2.18
7,7,Iskjaskst81,20.0,Male,162,Abyssal Shard,2.67
8,8,Undjask33,22.0,Male,21,Souleater,1.10
9,9,Chanosian48,35.0,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58


## Player Count

* Display the total number of players


In [329]:
# Create a data subset that we can use later that only looks at unique players
singlePlayer = purchase_data.drop_duplicates(subset="SN")

# Create separate df for just the player count
singleCount = pd.DataFrame({'Player Count':[singlePlayer['SN'].count()]})
singleCount

,Player Count
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [330]:
# Below we demonstrate a very basic extended way to get the data we are looking for
# Each value ends up as its own variable which we push into a dataframe at the end
items = purchase_data['Item Name'].drop_duplicates()
itemcount = items.count()

price = purchase_data['Price'].mean()

purchCount = purchase_data['Purchase ID'].count()
totalRevenue = purchase_data['Price'].sum()

purchAn_df = pd.DataFrame({'Number of Unique Items':[itemcount],'Average Price':[price],'Number of Purchases':[purchCount],'Total Revenue':[totalRevenue]})
# Change the format to money
purchAn_df['Average Price'] = purchAn_df['Average Price'].map('${:,.2f}'.format)
purchAn_df['Total Revenue'] = purchAn_df['Total Revenue'].map('${:,.2f}'.format)
purchAn_df.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [331]:
# Find unique gender count and then create a percentage
genderCount = pd.DataFrame(singlePlayer['Gender'].value_counts())
genderCount['Percentage of Players'] = ((genderCount['Gender'] / genderCount['Gender'].sum()) * 100).map('{:,.2f}%'.format)

genderCount.head()

,Gender,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [332]:
# Finally getting into the swing of things and realising there is an easier way to build dataframes
genderGroup = purchase_data.groupby('Gender') 
genderAn = pd.DataFrame(genderGroup['Purchase ID'].count()) 
genderAn.rename(columns={'Purchase ID':'Purchase Count'},inplace=True) 
genderAn['Average Purchase Price'] = genderGroup['Price'].mean().map('${:,.2f}'.format) 
genderAn['Total Purchase Value'] = genderGroup['Price'].sum().map('${:,.2f}'.format)
# work out the average per person by calling back to the singleUsers dataframe we created earlier
genderAn['Avg Total Purchase per Person'] = (genderGroup['Price'].sum() / singleUsers['Gender'].value_counts()).map('${:,.2f}'.format) 
genderAn

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [333]:
# Bins gave me some weird errors for a long time and it's where I got caught up on

bins = [0,9,14,19,24,29,34,39,100] # Create bin values - could explore float("inf") or .max()
labels = ["<10","10-14","15-19","20-24","25-29","30-34","35-39",">40"] # bin names
singlePlayer['Age'] = singlePlayer['Age'].astype(float) # really should move this up where we create this dataframe

# Add the Age Range bins onto the singlePlayer data frame. Could be it's own dataframe maybe?
singlePlayer['Age Range'] = pd.cut(singlePlayer['Age'], bins, labels=labels, include_lowest=True)

# Group on the age range and then work out player counts and percentages
ageGroup_df = singlePlayer.groupby('Age Range')
ageDemo_df = pd.DataFrame(ageGroup_df['Age'].count())
ageDemo_df.rename(columns={'Age':'Total Count'},inplace=True)
ageDemo_df['Percentage of Players'] = ((ageDemo_df['Total Count'] / singlePlayer['Age'].count()) *100).map('{:,.2f}%'.format)
ageDemo_df

,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
>40,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [335]:
# Instructions say to add the bins to the purchase_data dataframe. Maybe we could have created a completely separate one?
# Same analysis as previous steps

purchase_data['Age Ranges'] =  pd.cut(purchase_data['Age'], bins, labels=labels, include_lowest=True)
PurAnAge = purchase_data.groupby('Age Ranges')
purAnAge_df = pd.DataFrame(PurAnAge['Purchase ID'].count())
purAnAge_df.rename(columns={"Purchase ID":"Purchase Count"},inplace=True)
purAnAge_df['Average Purchase Price'] = PurAnAge['Price'].mean().map('${:,.2f}'.format)
purAnAge_df['Total Purchase Value'] = PurAnAge['Price'].sum().map('${:,.2f}'.format)
purAnAge_df['Avg Total Purchase per Person'] = (PurAnAge['Price'].sum() / ageDemo_df['Total Count']).map('${:,.2f}'.format)
purAnAge_df.head(10)


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
>40,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [285]:
# I really hate groupbys and indexes but I think I'm getting a hang of it now
# Had to build out the same analysis as previous however needed to remove index in order to sort by Total Purchase Value
topSpenders_grp = purchase_data.groupby('SN')
topSpenders_df = pd.DataFrame(topSpenders_grp['Purchase ID'].count())
topSpenders_df.rename(columns={"Purchase ID":"Purchase Count"},inplace=True)
topSpenders_df['Average Purchase Price'] = topSpenders_grp['Price'].mean().map('${:,.2f}'.format)
topSpenders_df['Total Purchase Value'] = topSpenders_grp['Price'].sum()

# Reset index in order to sort, then reset index
topSpenders_df = topSpenders_df.reset_index().sort_values(['Total Purchase Value'], ascending=False).set_index('SN')
topSpenders_df['Total Purchase Value'] = topSpenders_df['Total Purchase Value'].map('${:,.2f}'.format)
topSpenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [323]:
itemStats = purchase_data.filter(['Item ID','Item Name','Purchase ID','Price']) # Pull relevant fields from original dataframe
itemStats_grp = itemStats.groupby(['Item ID','Item Name'])
itemStats_df = pd.DataFrame(itemStats_grp['Purchase ID'].count())
itemStats_df.rename(columns={"Purchase ID":"Purchase Count"},inplace=True)
itemStats_df['Total Purchase Value'] = itemStats_grp['Price'].sum()
itemStats_df['Price'] = itemStats_df['Total Purchase Value'] / itemStats_df['Purchase Count'] # Not sure if this is the right way to do this, but the values match?
itemStats_df = itemStats_df[['Purchase Count','Price','Total Purchase Value']]

# Putting this into a separate dataframe because of the next question/task
popItems_df = itemStats_df
# sort by purchase count
popItems_df = popItems_df.reset_index().sort_values(['Purchase Count'], ascending=False).set_index(['Item ID','Item Name']) 
popItems_df['Total Purchase Value'] = popItems_df['Total Purchase Value'].map('${:,.2f}'.format)
popItems_df['Price'] = popItems_df['Price'].map('${:,.2f}'.format)

popItems_df.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [326]:
profItems_df = itemStats_df
# sort by purchase volume
profItems_df = profItems_df.reset_index().sort_values(['Total Purchase Value'], ascending=False).set_index(['Item ID','Item Name'])
profItems_df['Total Purchase Value'] = profItems_df['Total Purchase Value'].map('${:,.2f}'.format)
profItems_df['Price'] = profItems_df['Price'].map('${:,.2f}'.format)

profItems_df.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
